# Connect Playlists and Locations

In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_json('artists_shows.json', lines=True)

In [4]:
print(df.shape)
df.head(2)

(134943, 3)


,artist,date,loc
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"


In [7]:
df.columns = ['artist', 'date', 'location']    # rename loc -> location
domestics = df[df['location'].str.endswith('US')]
recent_domestics = domestics[domestics['date'] >= '2015']
locations = recent_domestics.copy()

In [9]:
locations['location'].unique().shape

(2023,)